![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FGenerate&file=Google+Gen+AI+Python+SDK.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Google%20Gen%20AI%20Python%20SDK.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGenerate%2FGoogle%2520Gen%2520AI%2520Python%2520SDK.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Google%20Gen%20AI%20Python%20SDK.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Generate/Google%20Gen%20AI%20Python%20SDK.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Google Gen AI Python SDK

The introduction of [Gemini 2.0](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2) brought a [new SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2#google-gen) that unifies the Gemini Developer API and the Gemini API on Vertex AI. It also supports the previous Gemini 1.5 models. More Background:
- the [Google Gen AI SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview)  
- The package `google-genai` for Python on GitHub [here](https://github.com/googleapis/python-genai)
- Companion workflow for Gemini using the Vertex AI SDK: [Vertex AI Gemini API](./Vertex%20AI%20Gemini%20API.ipynb)


This notebook briefly sets up the Vertex AI SDK and new Gen AI API and compares there usage for text and multimodal prompts.

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [2]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [3]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [4]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.71.0'),
    ('google.genai', 'google-genai'),
    ('pydantic', 'pydantic'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [8]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'gen-ai-sdk'

packages:

In [62]:
# standard packages
import os, shutil, enum, json, math

import IPython
import pydantic

# vertex ai imports
from google.cloud import aiplatform
import vertexai
import vertexai.generative_models # for Gemini Models

# google gen ai sdk
from google import genai

In [11]:
aiplatform.__version__

'1.78.0'

In [12]:
genai.__version__

'1.2.0'

clients:

In [13]:
vertexai.init(project = PROJECT_ID, location = REGION)
genai_client = genai.Client(vertexai = True, project = PROJECT_ID, location = REGION)

---
## Multimodal Inputs

Prompts can extend beyond text to include audio, video, images, and documents as context.  This means you can [design multimodal prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/design-multimodal-prompts) that include combinations of these file types as grounding for Gemini.

### Get The Documents

A set of example file inputs for multimodal prompts were created in this companion workflow: [Create Files For Multimodal Prompt Examples](./Create%20Files%20For%20Multimodal%20Prompt%20Examples.ipynb) and the outputs are already included in the repository.

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [14]:
local_dir = './files/multimodal-inputs'

In [15]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Generate/files/multimodal-inputs', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `./files/multimodal-inputs`


---
## Vertex AI Gemini API

In [16]:
system_instructions = 'Provide brief but descritive answer.'

In [17]:
vertex_gemini_pro = vertexai.generative_models.GenerativeModel(
    model_name = "gemini-1.5-pro-002",
    system_instruction = system_instructions
)
vertex_gemini_flash = vertexai.generative_models.GenerativeModel(
    model_name = "gemini-1.5-flash-002",
    system_instruction = system_instructions
)

### Text Prompts

In [18]:
prompt_text = 'What is the curve?'

In [19]:
vertex_gemini_pro.generate_content(
    contents = prompt_text,
    generation_config = vertexai.generative_models.GenerationConfig(
        temperature = 0.5
    )
).text

"A curve is a continuous and smooth line, often bending and changing direction, that isn't straight. It can be described mathematically by an equation.\n"

In [20]:
vertex_gemini_flash.generate_content(
    contents = prompt_text,
    generation_config = vertexai.generative_models.GenerationConfig(
        temperature = 0.5
    )
).text

'Please provide the equation or a description of the curve.  I need more information to tell you what it is.\n'

### Multimodal Prompts

#### Review The Image

In [21]:
IPython.display.Image(f"{local_dir}/plot.png")

#### Read The Image Data

In [22]:
with open(f"{local_dir}/plot.png", 'rb') as f:
    image_read = f.read()

#### Inline Image From Bytes

In [23]:
image_bytes = vertexai.generative_models.Image.from_bytes(
    data = image_read
)

In [24]:
image_bytes._mime_type

'image/png'

In [25]:
type(image_bytes)

vertexai.generative_models._generative_models.Image

#### Generate

In [26]:
vertex_gemini_pro.generate_content(
    contents = [image_bytes, prompt_text],
    generation_config = vertexai.generative_models.GenerationConfig(
        temperature = 0.5
    )
).text

'It appears to be a sine or cosine wave.  More specifically, it resembles a graph of sin(x).\n'

In [27]:
vertex_gemini_flash.generate_content(
    contents = [image_bytes, prompt_text],
    generation_config = vertexai.generative_models.GenerationConfig(
        temperature = 0.5
    )
).text

'The curve is a cosine function.  More specifically, it appears to be a cosine wave with a period of approximately 6 and an amplitude of 1.\n'

---
## Google Gen AI SDK

In [28]:
models = {
    "gemini": {
        "pro": {
            "1.5": "gemini-1.5-pro-002",
            "2.0": "gemini-2.0-pro-exp-02-05"
        },
        "flash": {
            "1.5": "gemini-1.5-flash-002",
            "2.0": "gemini-2.0-flash-001"
        }
    }
}

### Text Prompts

In [29]:
genai_client.models.generate_content(
    model = models['gemini']['pro']['1.5'],
    contents = prompt_text,
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'A curve is a continuous and smooth line, often bending and changing direction.  It can be described mathematically as a set of points whose coordinates satisfy a specific equation or function.\n'

In [30]:
genai_client.models.generate_content(
    model = models['gemini']['pro']['2.0'],
    contents = prompt_text,
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'A curve is a continuous and smooth line that deviates from straightness without sharp edges or corners. It can be in two dimensions (like a circle) or three dimensions (like a helix).\n'

In [31]:
genai_client.models.generate_content(
    model = models['gemini']['flash']['2.0'],
    contents = prompt_text,
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'In general, a curve is a one-dimensional, continuous object. More specifically, it can refer to:\n\n*   **A line that is not straight:** This is the most common understanding.\n*   **A graphical representation of a function:** Showing the relationship between two or more variables.\n*   **A mathematical object defined by an equation:** Such as a parabola or a circle.\n'

In [32]:
genai_client.models.generate_content(
    model = models['gemini']['flash']['1.5'],
    contents = prompt_text,
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'Please provide me with the equation, a graph, or a description of the curve.  I need more information to tell you what it is.\n'

### Multimodal Prompts

#### Review The Image

In [33]:
IPython.display.Image(f"{local_dir}/plot.png")

#### Read The Image Data

In [34]:
with open(f"{local_dir}/plot.png", 'rb') as f:
    image_read = f.read()

#### Inline Image From Bytes

In [35]:
image_bytes = genai.types.Part.from_bytes(
    data = image_read,
    mime_type = 'image/png'
)

In [36]:
type(image_bytes)

google.genai.types.Part

#### Generate

In [37]:
genai_client.models.generate_content(
    model = models['gemini']['pro']['1.5'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'It appears to be a sine or cosine wave.  More specifically, it resembles a sine wave shifted horizontally or a cosine wave.\n'

In [38]:
genai_client.models.generate_content(
    model = models['gemini']['pro']['2.0'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'Sine wave.\n'

In [39]:
genai_client.models.generate_content(
    model = models['gemini']['flash']['2.0'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

'The curve is a sine wave.\n'

In [40]:
genai_client.models.generate_content(
    model = models['gemini']['flash']['1.5'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0.3
    )
).text

"That's a plot of a cosine function, specifically it appears to be a cosine wave with a period of approximately 6.28 (which is close to 2π).\n"

### Controlled Prompts

#### With Enum For Classification

In [44]:
class Curves(str, enum.Enum):
    SINE = 'Sine'
    COSINE = 'Cosine'
    LINE = 'Line'
    TANGENT = 'Tangent'
    COTANGENT = 'Cotangent'
    SECANT = 'Secant'
    EXPONENTIAL = 'Exponential'
    LOGARITHMIC = 'Logarithmic'

In [45]:
genai_client.models.generate_content(
    model = models['gemini']['flash']['2.0'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0,
        response_mime_type = 'text/x.enum',
        response_schema = Curves
    )
).text

'Sine'

#### With JSON Including List of ENUM - Using Pydantic

In [47]:
class MultiCurve(pydantic.BaseModel):
    curves: typing.List[Curves]

In [50]:
response = genai_client.models.generate_content(
    model = models['gemini']['flash']['2.0'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0,
        response_mime_type = 'application/json',
        response_schema = MultiCurve
    )
)
response.text

'{\n  "curves": [\n    "Sine"\n  ]\n}'

In [52]:
json.loads(response.text)

{'curves': ['Sine']}

#### Review and Validate JSON - With Pydantic

In [53]:
MultiCurve.model_validate_json(response.text)

MultiCurve(curves=[<Curves.SINE: 'Sine'>])

In [54]:
MultiCurve.model_validate(json.loads(response.text))

MultiCurve(curves=[<Curves.SINE: 'Sine'>])

In [55]:
try:
    print(MultiCurve.model_validate(json.loads(response.text)))
except pydantic.ValidationError as e:
    print(e.errors)

curves=[<Curves.SINE: 'Sine'>]


### Understanding Responses With Token Probabilities

We know 'Sine' is correct but what was the second best guess?  Or if it was wrong we might want to know how close was the second best guess and what was it?

This is where reviewing the responses first token and its alternatives can help out.
- `response_logprobs = True` enable the token level details in the output along with the log probability of each token
- `logprobs = 5` enables the inclusion of the top token candidates (chosen plus alternatives) at each generate step

Here we enable the token level details and request the top 5 (max) tokens at each step.  By reviewing the first step we can understand how the choice of the enum played out

In [57]:
response = genai_client.models.generate_content(
    model = models['gemini']['flash']['2.0'],
    contents = [image_bytes, prompt_text],
    config = genai.types.GenerateContentConfig(
        system_instruction = system_instructions,
        temperature = 0,
        response_mime_type = 'text/x.enum',
        response_schema = Curves,
        response_logprobs = True,
        logprobs = 5
    )
)
response.text

'Sine'

In [58]:
response.candidates[0].logprobs_result

LogprobsResult(chosen_candidates=[LogprobsResultCandidate(log_probability=-0.000106890686, token='Sine', token_id=None)], top_candidates=[LogprobsResultTopCandidates(candidates=[LogprobsResultCandidate(log_probability=-0.000106890686, token='Sine', token_id=None), LogprobsResultCandidate(log_probability=-9.188898, token=' Sine', token_id=None), LogprobsResultCandidate(log_probability=-11.712154, token='Cosine', token_id=None), LogprobsResultCandidate(log_probability=-15.362929, token='Line', token_id=None), LogprobsResultCandidate(log_probability=-15.744335, token='S', token_id=None)])])

In [59]:
response.candidates[0].logprobs_result.chosen_candidates

[LogprobsResultCandidate(log_probability=-0.000106890686, token='Sine', token_id=None)]

In [60]:
response.candidates[0].logprobs_result.top_candidates

[LogprobsResultTopCandidates(candidates=[LogprobsResultCandidate(log_probability=-0.000106890686, token='Sine', token_id=None), LogprobsResultCandidate(log_probability=-9.188898, token=' Sine', token_id=None), LogprobsResultCandidate(log_probability=-11.712154, token='Cosine', token_id=None), LogprobsResultCandidate(log_probability=-15.362929, token='Line', token_id=None), LogprobsResultCandidate(log_probability=-15.744335, token='S', token_id=None)])]

In [65]:
def candidates(response):
    markdown_table = "|Token Order| Top Candidates | Log Probability | Probability |\n"
    markdown_table += "|---|---|---|---|\n"
    logprobs_result = response.candidates[0].logprobs_result
    for top_candidates_group in logprobs_result.top_candidates:
        for candidate in top_candidates_group.candidates:
            token = candidate.token
            logprob = f"{candidate.log_probability:.6f}"
            # Handle potential underflow
            prob = f"{math.exp(candidate.log_probability):.6f}" if candidate.log_probability > -700 else "0.000000"
            markdown_table += f"|| {token} | {logprob} | {prob} |\n"

    IPython.display.display(IPython.display.Markdown(markdown_table))
    
candidates(response)

|Token Order| Top Candidates | Log Probability | Probability |
|---|---|---|---|
|| Sine | -0.000107 | 0.999893 |
||  Sine | -9.188898 | 0.000102 |
|| Cosine | -11.712154 | 0.000008 |
|| Line | -15.362929 | 0.000000 |
|| S | -15.744335 | 0.000000 |
